# Quantum Detector Tomography

This notebook contains examples for using QDT module with Qiskit. test_utilities module, with methods used only in tests, shall be used.

In [1]:
import povmtools
import ancillary_functions as anf
import numpy as np

from qiskit import IBMQ, Aer, execute
from qiskit.providers.aer import noise

from quantum_tomography_qiskit import detector_tomography_circuits
from DetectorTomographyFitter import DetectorTomographyFitter

## Single Qubit detector tomography example

In below example it's shown how to implement single-qubit Quantum Detector Tomography (QDT) using our module. 

First we need to create quantum circuits which will be used to implement QDT. To do so, need to define indices of qubits on which we want to perform tomography. In this example we choose single qubit with label 3.

In [2]:
# choose qubit indices
test_qubit_indices = [3]

We will also need kets upon which the tomography shall be based. In this scenario we shall use overcomplete set of Pauli's eigen states. Our module provides them in following way, using povmtools module.

In [3]:
test_probe_kets = povmtools.pauli_probe_eigenkets

Now we call detector_tomography_circuits method, which will generate desired circuits. 

In [4]:
test_circuits = detector_tomography_circuits(test_qubit_indices, test_probe_kets)

### User-defined probe states

User can define list of single-qubit state vectors which will be used to perform tomography of the detector (in the case of multi-qubit tomography, only single-qubit set is required - the multi-qubit states are constructed from proper tensor products of those, see multi-qubit QDT example below).

In order for tomography to work properly, the set of passed qubit state vectors, when mapped onto quantum states (i.e., density matrices), must span the space of Hermitian matrices. In other words, it must form an operator basis (it might be an overcomplete basis).

If no probe_kets are provided then overcomplete set of Pauli's eigen states is used:

{|0>, |1>, |X+>, |X->, |Y+>, |Y->}

### Implementation of QDT

After defining tomography circuits, we need to implement them. 

To do so, we use Qiskit simulator in the following way.

In [5]:
backend = Aer.get_backend('qasm_simulator')

# No noise
QDT_job = execute(test_circuits, backend=backend, shots=8192)

# With noise
#QDT_job = execute(test_circuits, backend=backend, shots=8192, noise_model=noise_model)

results = QDT_job.result()

Automatic discovery of qconfig credentials failed: "Error loading Qconfig.py: module 'Qconfig' has no attribute 'APItoken'"


With probe kets and job results we can begin to perform QDT. We start with instantiating DetectorTomographyFitter class object.

In [6]:
DTF = DetectorTomographyFitter()

This class contains method called get_maximum_likelihood_povm_estimator which returns the classical description of our detector. This methods requires results and probe kets as arguments. We call the method and print the results.

In [9]:
ml_povm_estimator = DTF.get_maximum_likelihood_povm_estimator(results, test_probe_states)

print(ml_povm_estimator)

[array([[ 9.99977275e-01+9.71445147e-17j, -2.44132332e-04-4.76079572e-03j],
       [-2.44132332e-04+4.76079572e-03j,  2.27252929e-05-6.77626358e-20j]]), array([[2.27252929e-05-3.25260652e-19j, 2.44132332e-04+4.76079572e-03j],
       [2.44132332e-04-4.76079572e-03j, 9.99977275e-01-1.38777878e-17j]])]


## Multiple qubit case

In the mutli-qubit QDT scenario, the only difference is using our approach is using desired qubit indices. Providing following qubit indices will return QDT for measurement on qubits 3 and 1 (considered as one measurement).

In [ ]:
# choose qubit indices
test_qubit_indices = [3, 1]

The order of qubits is not important, as **the returned estimator is always for qubits given in ascending order**.